In [2]:
from elasticsearch import Elasticsearch
import config

In [3]:
es = Elasticsearch(
    hosts=config.HOSTS,
    basic_auth=config.HTTP_AUTH,
    ca_certs=config.CA_CERTS,
    verify_certs=config.VERIFY_CERTS)

## Prepare the data

In [4]:
import pandas as pd

df = pd.read_csv("myntra_products_catalog.csv").loc[:499]
df.head()


,ProductID,ProductName,ProductBrand,Gender,Price (INR),NumImages,Description,PrimaryColor
0,10017413,DKNY Unisex Black & Grey Printed Medium Trolle...,DKNY,Unisex,11745,7,"Black and grey printed medium trolley bag, sec...",Black
1,10016283,EthnoVogue Women Beige & Grey Made to Measure ...,EthnoVogue,Women,5810,7,Beige & Grey made to measure kurta with churid...,Beige
2,10009781,SPYKAR Women Pink Alexa Super Skinny Fit High-...,SPYKAR,Women,899,7,Pink coloured wash 5-pocket high-rise cropped ...,Pink
3,10015921,Raymond Men Blue Self-Design Single-Breasted B...,Raymond,Men,5599,5,Blue self-design bandhgala suitBlue self-desig...,Blue
4,10017833,Parx Men Brown & Off-White Slim Fit Printed Ca...,Parx,Men,759,5,"Brown and off-white printed casual shirt, has ...",White


In [5]:
df.isna().value_counts()

ProductID  ProductName  ProductBrand  Gender  Price (INR)  NumImages  Description  PrimaryColor
False      False        False         False   False        False      False        False           468
                                                                                   True             32
Name: count, dtype: int64

In [6]:
df.fillna("None", inplace=True)

## Convert the relevant field to Vector using BERT Model

In [7]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("all-mpnet-base-v2")

In [8]:
df["DescriptionVector"] = df["Description"].apply(lambda x: model.encode(x))

## Create new index in Elasticsearch

In [13]:
from index_mapping import index_mapping

es.indices.create(index="all_products", mappings=index_mapping)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'all_products'})

## Ingest the data into index

In [9]:
record_list = df.to_dict("records")

In [17]:
for record in record_list:
    try:
        es.index(index="all_products", document=record, id=record["ProductID"])
    except Exception as e:
        print(e)

## Search the data

In [13]:
input_keyword = "Blue Shoes"
vector_of_input_keyword = model.encode(input_keyword)


query = {
    "field": "DescriptionVector",
    "query_vector": vector_of_input_keyword,
    "k": 2,
    "num_candidates": 500,
}

res = es.knn_search(index="all_products", knn=query, source=["ProductName", "Description"])
res["hits"]["hits"]

C:\Users\Aaron\AppData\Local\Temp\ipykernel_4560\661817962.py:12: ElasticsearchWarning: The kNN search API has been replaced by the `knn` option in the search API.
  res = es.knn_search(index="all_products", knn=query, source=["ProductName", "Description"])


[{'_index': 'all_products',
  '_id': '10018013',
  '_score': 0.6142942,
  '_source': {'ProductName': 'Puma Men Blue Sneakers',
   'Description': 'A pair of round-toe blue sneakers, has regular styling, lace-up detailTextile upperCushioned footbedTextured and patterned outsoleWarranty: 3 monthsWarranty provided by brand/manufacturer'}},
 {'_index': 'all_products',
  '_id': '10018075',
  '_score': 0.6142942,
  '_source': {'ProductName': 'Puma Men Blue Sneakers',
   'Description': 'A pair of round-toe blue sneakers, has regular styling, lace-up detailTextile upperCushioned footbedTextured and patterned outsoleWarranty: 3 monthsWarranty provided by brand/manufacturer'}}]